<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../module1-text-data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv')

In [34]:
df.tail(2)

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs,tokens
28330,AVqkIhxunnc1JgDc3kg_,2017-03-06T14:59:43Z,2019-02-23T02:49:38Z,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",B018T075DC,Amazon,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",Electronics,https://www.upccodesearch.com/images/barcode/0...,"amazon/b018t075dc,firehd8tabletwithalexa8hddis...",...,True,NaN,0.0,5,http://reviews.bestbuy.com/3545/5620410/review...,This Tablet does absolutely everything I want!...,You get the entire World for less than $100!,SandyJ,http://reviews.bestbuy.com/3545/5620410/review...,"[this, tablet, absolutely, want!, watch, tv, s..."
28331,AVqkIhxunnc1JgDc3kg_,2017-03-06T14:59:43Z,2019-02-23T02:49:38Z,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",B018T075DC,Amazon,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",Electronics,https://www.upccodesearch.com/images/barcode/0...,"amazon/b018t075dc,firehd8tabletwithalexa8hddis...",...,True,NaN,0.0,4,http://reviews.bestbuy.com/3545/5620410/review...,"At ninety dollars, the expectionations are low...",You get what your paying for,Hung,http://reviews.bestbuy.com/3545/5620410/review...,"[at, ninety, dollars,, expectionations, low,, ..."


In [4]:
reviews = df['reviews.text'].tolist()

In [22]:
len(reviews)

28332

# Fit LDA Model

## Pre-Processing

In [5]:
import spacy
from spacy.tokenizer import Tokenizer

In [6]:
nlp = spacy.load("en_core_web_lg")

In [7]:
tokenizer = Tokenizer(nlp.vocab)

In [8]:
tokens = []

In [9]:
STOP_WORDS = nlp.Defaults.stop_words.union(['I', 'amazon', 'i', 'Amazon', 'it', "it's", 'it.', 'the', 'this', 'them', 'of', 'a'])

In [10]:
for doc in tokenizer.pipe(df['reviews.text'], batch_size = 500):
    # Initialize doc tokens
    doc_tokens = []
    
    # Filter for stop words
    for token in doc:
        if token.text not in STOP_WORDS:
            doc_tokens.append(token.text.lower())
            
    # Append doc_tokens to tokens
    tokens.append(doc_tokens)

In [11]:
df['tokens'] = tokens

In [12]:
df['tokens'].head()

0    [order, 3, item, bad, quality., is, missing, b...
1               [bulk, expensive, way, products, like]
2                      [well, duracell, price, happy.]
3        [seem, work, brand, batteries, better, price]
4     [these, batteries, long, lasting, price, great.]
Name: tokens, dtype: object

## LDA

In [13]:
from gensim import corpora
from gensim.models.ldamulticore import LdaMulticore

In [15]:
id2word = corpora.Dictionary(tokens)

In [23]:
# Filter extreme values
id2word.filter_extremes(no_below=40, no_above=0.90)

In [25]:
# Create corpus
corpus = [id2word.doc2bow(text) for text in tokens]

In [ ]:
# Instantiate LDA
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   num_topics=3,
                   passes=10,
                   workers=6)

In [42]:
lda.print_topics()

[(0,
  '0.023*"the" + 0.022*"tablet" + 0.020*"kindle" + 0.016*"fire" + 0.015*"like" + 0.013*"great" + 0.013*"this" + 0.012*"it" + 0.012*"screen" + 0.011*"use"'),
 (1,
  '0.029*"tablet" + 0.029*"great" + 0.025*"loves" + 0.024*"easy" + 0.024*"love" + 0.023*"bought" + 0.021*"my" + 0.020*"old" + 0.019*"kids" + 0.019*"year"'),
 (2,
  '0.047*"batteries" + 0.047*"great" + 0.042*"good" + 0.024*"price" + 0.019*"long" + 0.018*"work" + 0.014*"buy" + 0.014*"they" + 0.014*"price." + 0.013*"batteries."')]

## Visualize

In [43]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [44]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.033016 -0.120419       1        1  37.586887
2     -0.212089  0.044963       2        1  33.750820
1      0.179073  0.075455       3        1  28.662285, topic_info=     Category         Freq        Term        Total  loglift  logprob
17    Default  4682.000000   batteries  4682.000000  30.0000  30.0000
1043  Default  2122.000000       loves  2122.000000  29.0000  29.0000
63    Default  5525.000000        good  5525.000000  28.0000  28.0000
107   Default  1579.000000        year  1579.000000  27.0000  27.0000
663   Default  1752.000000         old  1752.000000  26.0000  26.0000
257   Default  1942.000000          my  1942.000000  25.0000  25.0000
637   Default  2786.000000        easy  2786.000000  24.0000  24.0000
1144  Default  5007.000000      tablet  5007.000000  23.0000  23.0000
21    Default  1941.000000        work  1941.000000  22.0000  22.0000
1086  Default  2434.000000      kindle  2434.000000  21.0000  21.0000
192   Default  1891.000000        kids  1891.000000  20.0000  20.0000
24    Default  2062.000000        long  2062.000000  19.0000  19.0000
120   Default  1283.000000  batteries.  1283.000000  18.0000  18.0000
15    Default  3014.000000       price  3014.000000  17.0000  17.0000
25    Default  1154.000000       these  1154.000000  16.0000  16.0000
73    Default  8543.000000       great  8543.000000  15.0000  15.0000
29    Default  3020.000000      bought  3020.000000  14.0000  14.0000
909   Default   951.000000         she   951.000000  13.0000  13.0000
95    Default  3336.000000        love  3336.000000  12.0000  12.0000
660   Default  2025.000000        fire  2025.000000  11.0000  11.0000
1003  Default  1336.000000      screen  1336.000000  10.0000  10.0000
19    Default   891.000000       brand   891.000000   9.0000   9.0000
714   Default   770.000000        gift   770.000000   8.0000   8.0000
133   Default  1651.000000        they  1651.000000   7.0000   7.0000
59    Default  1641.000000      price.  1641.000000   6.0000   6.0000
220   Default  2260.000000          it  2260.000000   5.0000   5.0000
2     Default  1808.000000     battery  1808.000000   4.0000   4.0000
562   Default  1145.000000       games  1145.000000   3.0000   3.0000
889   Default   714.000000    daughter   714.000000   2.0000   2.0000
622   Default   730.000000         son   730.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
931    Topic3   349.812103         kid   369.718262   1.1942  -5.4806
33     Topic3   575.512695   christmas   617.874390   1.1786  -4.9827
1117   Topic3   181.388275  christmas.   188.483490   1.2112  -6.1373
899    Topic3   230.349686    children   241.354065   1.2029  -5.8984
192    Topic3  1627.494629        kids  1891.609375   1.0992  -3.9432
723    Topic3   202.085510     durable   213.053513   1.1967  -6.0293
715    Topic3   170.651733       gift.   179.033920   1.2016  -6.1983
503    Topic3   447.945892         it!   510.468231   1.1189  -5.2333
637    Topic3  2054.073975        easy  2786.724854   0.9445  -3.7104
354    Topic3   291.675629       child   324.594482   1.1427  -5.6623
562    Topic3   875.759460       games  1145.286255   0.9813  -4.5629
29     Topic3  1969.178955      bought  3020.300293   0.8218  -3.7526
585    Topic3   608.776306         set   775.523926   1.0075  -4.9265
95     Topic3  2046.773315        love  3336.736328   0.7609  -3.7139
389    Topic3   649.729858        use.   852.410645   0.9781  -4.8614
730    Topic3   333.645905       loved   387.046051   1.1011  -5.5279
1144   Topic3  2450.752441      tablet  5007.008789   0.5351  -3.5338
114    Topic3   637.280029     perfect   918.951660   0.8836  -4.8807
217    Topic3   592.219727           2   864.493713   0.8713  -4.9541
128    Topic3   737.664612   purchased  1247.711792   0.7240  -4.7345
73     Topic3  2426.973877       great  8543.11

In [ ]:
# Instantiate LDA
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   num_topics=2,
                   passes=10,
                   workers=6)

In [38]:
lda.print_topics()

[(0,
  '0.029*"tablet" + 0.020*"great" + 0.016*"easy" + 0.016*"the" + 0.015*"this" + 0.015*"bought" + 0.014*"love" + 0.014*"use" + 0.013*"kindle" + 0.013*"loves"'),
 (1,
  '0.041*"great" + 0.035*"batteries" + 0.028*"good" + 0.019*"price" + 0.015*"long" + 0.015*"buy" + 0.014*"work" + 0.013*"they" + 0.012*"battery" + 0.010*"love"')]

## Visualize

In [39]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [40]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
0      0.140513  0.0       1        1  56.636112
1     -0.140513  0.0       2        1  43.363892, topic_info=     Category         Freq         Term        Total  loglift  logprob
17    Default  4402.000000    batteries  4402.000000  30.0000  30.0000
1144  Default  4804.000000       tablet  4804.000000  29.0000  29.0000
24    Default  2022.000000         long  2022.000000  28.0000  28.0000
133   Default  1723.000000         they  1723.000000  27.0000  27.0000
21    Default  1837.000000         work  1837.000000  26.0000  26.0000
15    Default  2924.000000        price  2924.000000  25.0000  25.0000
637   Default  2785.000000         easy  2785.000000  24.0000  24.0000
63    Default  5085.000000         good  5085.000000  23.0000  23.0000
1043  Default  2174.000000        loves  2174.000000  22.0000  22.0000
31    Default  2147.000000          buy  2147.000000  21.0000  21.0000
1086  Default  2273.000000       kindle  2273.000000  20.0000  20.0000
120   Default  1206.000000   batteries.  1206.000000  19.0000  19.0000
73    Default  8508.000000        great  8508.000000  18.0000  18.0000
25    Default  1086.000000        these  1086.000000  17.0000  17.0000
663   Default  1792.000000          old  1792.000000  16.0000  16.0000
2     Default  1818.000000      battery  1818.000000  15.0000  15.0000
1195  Default  1396.000000      tablet.  1396.000000  14.0000  14.0000
19    Default   838.000000        brand   838.000000  13.0000  13.0000
964   Default  1298.000000         apps  1298.000000  12.0000  12.0000
146   Default  2766.000000         this  2766.000000  11.0000  11.0000
562   Default  1145.000000        games  1145.000000  10.0000  10.0000
107   Default  1640.000000         year  1640.000000   9.0000   9.0000
660   Default  1944.000000         fire  1944.000000   8.0000   8.0000
1003  Default  1242.000000       screen  1242.000000   7.0000   7.0000
909   Default   972.000000          she   972.000000   6.0000   6.0000
59    Default  1516.000000       price.  1516.000000   5.0000   5.0000
344   Default   671.000000        value   671.000000   4.0000   4.0000
220   Default  2244.000000           it  2244.000000   3.0000   3.0000
526   Default  1064.000000      quality  1064.000000   2.0000   2.0000
117   Default  1402.000000        don't  1402.000000   1.0000   1.0000
...       ...          ...          ...          ...      ...      ...
44     Topic2   289.274384         will   296.425262   0.8111  -6.0846
31     Topic2  1908.520508          buy  2147.320312   0.7177  -4.1979
261    Topic2   285.173615  performance   294.318268   0.8040  -6.0989
15     Topic2  2430.506836        price  2924.845703   0.6504  -3.9561
2      Topic2  1528.346069      battery  1818.034180   0.6620  -4.4201
58     Topic2   268.888214        power   277.791962   0.8030  -6.1577
526    Topic2   907.500732      quality  1064.112305   0.6763  -4.9413
210    Topic2   647.692078         i've   732.530518   0.7125  -5.2786
69     Topic2   465.679413       buying   510.578827   0.7435  -5.6085
63     Topic2  3609.341797         good  5085.732910   0.4926  -3.5607
89     Topic2   510.512634        them.   573.213013   0.7197  -5.5166
368    Topic2   405.192749         beat   442.025299   0.7485  -5.7476
59     Topic2  1196.803467       price.  1516.872803   0.5985  -4.6646
117    Topic2  1103.846680        don't  1402.023926   0.5964  -4.7454
73     Topic2  5178.390625        great  8508.662109   0.3390  -3.1997
401    Topic2   479.047485         deal   542.759888   0.7107  -5.5802
194    Topic2   650.423950          far   826.285461   0.5962  -5.2744
243    Topic2   660.269104           so   883.942871   0.5438  -5.2593
292    Topic2   931.915527         best  1489.420166   0.3666  -4.9148
268    Topic2   882.596497      product  1384.694458   0.3852  -4.9691
347    Topic2   535.006897         life   694.716614   0.5743  -5.469

This set of reviews seems to contain only reviews about batteries and tablets. Originally, I had split this up into 10 groups and there was a lot of overlap in the visualization. I brought it down to three and two groups were clustered together, both with 'tablet' in the top two words.

## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling